# Генеративные модели: Семинар 1 - Латентные пространства

## Цели семинара
- Изучить различные метрики качества генерации (LPIPS)
- Познакомиться с Deep Image Prior
- Освоить условную генерацию и манипуляции в латентном пространстве

## Предварительные требования
- PyTorch
- LPIPS
- torchvision
- PIL
- matplotlib

In [ ]:
import os
import logging
import numpy as np
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm, trange
from matplotlib import pyplot as plt
from IPython.display import display, HTML
import ipywidgets as widgets

from PIL import Image
from torchvision.utils import make_grid
from torchvision.transforms import ToPILImage, CenterCrop, ToTensor, Resize, GaussianBlur

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Устанавливаем сид для воспроизводимости
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Конфигурация
CONFIG = {
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'batch_size': 8,
    'n_steps': 2000,
    'truncation': 10,
    'lpips_weight': 0.5,
    'save_dir': Path('results')
}

# Создаем директорию для результатов
CONFIG['save_dir'].mkdir(exist_ok=True)

In [ ]:
def setup_environment(backend='Colab'):
    """Настройка окружения и загрузка необходимых файлов.
    
    Args:
        backend (str): Тип окружения ('Colab' или 'Local')
    """
    try:
        if backend == 'Colab':
            !git clone https://github.com/yandexdataschool/deep_vision_and_graphics.git
            !sudo apt install -y ninja-build
            %cd /content/deep_vision_and_graphics/week09_gans
            !wget https://www.dropbox.com/s/2kpsomtla61gjrn/pretrained.tar
            !tar -xvf pretrained.tar
        logger.info(f'Successfully set up {backend} environment')
    except Exception as e:
        logger.error(f'Failed to setup environment: {str(e)}')
        raise

setup_environment()

## 1. Метрики качества генерации

### 1.1 LPIPS (Learned Perceptual Image Patch Similarity)

LPIPS - метрика, которая использует особенности, извлеченные предобученной нейронной сетью, для оценки перцептивного сходства между изображениями. В отличие от традиционных метрик (MSE, PSNR), LPIPS лучше коррелирует с человеческим восприятием качества изображений.

In [ ]:
class ImageProcessor:
    """Класс для обработки и визуализации изображений."""
    
    def __init__(self, device=CONFIG['device']):
        self.device = device
        self.lpips_model = lpips.LPIPS('alexnet').to(device)
        
    def load_and_preprocess(self, image_path, size=256):
        """Загрузка и предобработка изображения.
        
        Args:
            image_path (str): Путь к изображению
            size (int): Размер выходного изображения
            
        Returns:
            torch.Tensor: Нормализованный тензор изображения
        """
        try:
            img = CenterCrop(size)(Resize(size)(Image.open(image_path)))
            tensor = ToTensor()(img)[:3]
            return 2 * tensor.unsqueeze(0).to(self.device) - 1
        except Exception as e:
            logger.error(f'Failed to load image: {str(e)}')
            raise
            
    def compare_images(self, ref_img, modified_images, titles=None):
        """Сравнение референсного изображения с модифицированными версиями.
        
        Args:
            ref_img (torch.Tensor): Референсное изображение
            modified_images (list): Список модифицированных изображений
            titles (list): Список заголовков для изображений
        """
        n_images = len(modified_images) + 1
        _, axs = plt.subplots(1, n_images, figsize=(4*n_images, 4), dpi=100)
        
        for ax in axs:
            ax.axis('off')
        
        # Показываем референсное изображение
        axs[0].imshow(to_image(ref_img))
        axs[0].set_title('Reference')
        
        # Показываем модифицированные изображения с LPIPS метрикой
        for i, img in enumerate(modified_images, 1):
            lpips_score = self.lpips_model(img, ref_img).item()
            axs[i].imshow(to_image(img))
            title = f'{titles[i-1]}\nLPIPS: {lpips_score:.3f}' if titles else f'LPIPS: {lpips_score:.3f}'
            axs[i].set_title(title)
            
        plt.tight_layout()

# Создаем интерактивные виджеты для экспериментов с искажениями
def create_distortion_widgets():
    blur_sigma = widgets.FloatSlider(value=2.0, min=0.1, max=5.0, step=0.1, description='Blur σ:')
    noise_std = widgets.FloatSlider(value=0.3, min=0.1, max=1.0, step=0.1, description='Noise σ:')
    return blur_sigma, noise_std

processor = ImageProcessor()
blur_sigma, noise_std = create_distortion_widgets()

def update_comparison(blur_sigma, noise_std):
    ref_img = processor.load_and_preprocess('sample.png')
    
    # Создаем искаженные версии
    img_blured = processor.normalize(ToTensor()(GaussianBlur(5, sigma=(blur_sigma, blur_sigma))(img))[:3])
    img_noised = ref_img + noise_std * torch.randn_like(ref_img)
    
    processor.compare_images(ref_img, [img_blured, img_noised], ['Blurred', 'Noised'])

widgets.interactive(update_comparison, blur_sigma=blur_sigma, noise_std=noise_std)

## 2. Deep Image Prior

Deep Image Prior (DIP) - это техника, которая использует структуру нейронной сети как prior для задач обработки изображений. В отличие от классических методов, DIP не требует предварительного обучения на большом наборе данных.

In [ ]:
class DeepImagePrior:
    """Реализация Deep Image Prior с визуализацией процесса оптимизации."""
    
    def __init__(self, config=CONFIG):
        self.config = config
        self.device = config['device']
        self.results = []
        
    def optimize(self, ref_img, lpips_weight, mask, callback=None):
        """Оптимизация генеративной модели.
        
        Args:
            ref_img (torch.Tensor): Референсное изображение
            lpips_weight (float): Вес LPIPS loss
            mask (torch.Tensor): Маска для оптимизации
            callback (callable): Функция обратного вызова для визуализации
            
        Returns:
            tuple: (оптимизированное изображение, модель)
        """
        G = deepcopy(G_ref)
        G.to(self.device).train()
        
        optimizer = torch.optim.Adam(G.parameters())
        mse = nn.MSELoss()
        lpips_model = lpips.LPIPS('alexnet').to(self.device)
        
        # Progress bar с метриками
        pbar = tqdm(range(self.config['n_steps']))
        for step in pbar:
            G.zero_grad()
            rec = G(z)
            
            # Вычисляем потери
            mse_loss = mse(mask * rec, mask * ref_img)
            lpips_loss = lpips_model(mask * rec, mask * ref_img)
            total_loss = (1.0 - lpips_weight) * mse_loss + lpips_weight * lpips_loss
            
            total_loss.backward()
            optimizer.step()
            
            # Обновляем progress bar
            pbar.set_description(f'MSE: {mse_loss.item():.4f}, LPIPS: {lpips_loss.item():.4f}')
            
            # Сохраняем промежуточные результаты
            if step % 100 == 0:
                self.results.append({
                    'step': step,
                    'image': rec.detach().cpu(),
                    'mse_loss': mse_loss.item(),
                    'lpips_loss': lpips_loss.item()
                })
                if callback:
                    callback(self.results[-1])
                    
        return rec, G
    
    def visualize_progress(self):
        """Визуализация процесса оптимизации."""
        n_samples = len(self.results)
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
        
        # График потерь
        mse_losses = [r['mse_loss'] for r in self.results]
        lpips_losses = [r['lpips_loss'] for r in self.results]
        
        ax1.plot(steps, mse_losses, 'b-', label='MSE Loss')
        ax1.plot(steps, lpips_losses, 'r-', label='LPIPS Loss')
        ax1.set_xlabel('Steps')
        ax1.set_ylabel('Loss')
        ax1.legend()
        ax1.grid(True)
        
        # Визуализация изображений
        selected_steps = np.linspace(0, len(self.results)-1, 5).astype(int)
        images = [self.results[i]['image'] for i in selected_steps]
        ax2.imshow(to_image_grid(torch.cat(images), nrow=len(images)))
        ax2.axis('off')
        ax2.set_title('Progression of Image Generation')
        
        plt.tight_layout()
        return fig


# Создаем интерактивный интерфейс для Deep Image Prior
def create_dip_interface():
    lpips_weight = widgets.FloatSlider(
        value=0.5,
        min=0.0,
        max=1.0,
        step=0.1,
        description='LPIPS Weight:'
    )
    
    mask_size = widgets.IntRangeSlider(
        value=[100, 150],
        min=0,
        max=256,
        step=10,
        description='Mask Range:'
    )
    
    run_button = widgets.Button(description='Run Optimization')
    output = widgets.Output()
    
    return lpips_weight, mask_size, run_button, output

### Демонстрация работы Deep Image Prior

In [ ]:
dip = DeepImagePrior()
lpips_weight, mask_size, run_button, output = create_dip_interface()

def run_optimization(_):
    with output:
        output.clear_output()
        
        # Создаем маску
        mask = torch.ones_like(ref_img)
        mask[:, :, mask_size.value[0]:mask_size.value[1], 
             mask_size.value[0]:mask_size.value[1]] = 0.0
        
        # Запускаем оптимизацию
        rec, G = dip.optimize(ref_img, lpips_weight.value, mask)
        
        # Показываем результаты
        dip.visualize_progress()
        plt.show()

run_button.on_click(run_optimization)

# Отображаем интерфейс
widgets.VBox([lpips_weight, mask_size, run_button, output])

## 3. Условная генерация

В этом разделе мы рассмотрим условную генерацию изображений с помощью BigGAN и изучим влияние различных параметров на качество генерации.

In [ ]:
class ConditionalGeneration:
    """Класс для экспериментов с условной генерацией."""
    
    def __init__(self, model_path='pretrained/G_ema.pth', img_size=128):
        self.G = make_big_gan(model_path, img_size).cuda().eval()
        
    def generate_with_classes(self, z, classes):
        """Генерация изображений с заданными классами.
        
        Args:
            z (torch.Tensor): Латентные векторы
            classes (torch.Tensor): Индексы классов
            
        Returns:
            torch.Tensor: Сгенерированные изображения
        """
        with torch.no_grad():
            cl_embed = self.G.big_gan.shared(classes)
            return self.G.big_gan(z, cl_embed)
    
    def interpolate_classes(self, z, class1, class2, steps=5):
        """Интерполяция между двумя классами.
        
        Args:
            z (torch.Tensor): Латентный вектор
            class1 (int): Первый класс
            class2 (int): Второй класс
            steps (int): Количество шагов интерполяции
            
        Returns:
            torch.Tensor: Интерполированные изображения
        """
        with torch.no_grad():
            cl1_embed = self.G.big_gan.shared(torch.tensor([class1]).cuda())
            cl2_embed = self.G.big_gan.shared(torch.tensor([class2]).cuda())
            
            alphas = torch.linspace(0, 1, steps).cuda()
            embeddings = torch.stack([
                torch.lerp(cl1_embed[0], cl2_embed[0], alpha)
                for alpha in alphas
            ])
            
            return self.G.big_gan(z.repeat(steps, 1), embeddings)
            
    def generate_with_truncation(self, num_samples, class_idx, truncation=1.0):
        """Генерация с усечением латентного пространства.
        
        Args:
            num_samples (int): Количество сэмплов
            class_idx (int): Индекс класса
            truncation (float): Параметр усечения
            
        Returns:
            torch.Tensor: Сгенерированные изображения
        """
        with torch.no_grad():
            tr = truncnorm(-truncation, truncation)
            z = torch.from_numpy(
                tr.rvs(num_samples * 512)
            ).view([num_samples, 512]).float().cuda()
            
            classes = torch.full([num_samples], class_idx, dtype=torch.int64).cuda()
            return self.generate_with_classes(z, classes)

### Интерактивные эксперименты с условной генерацией

In [ ]:
def create_conditional_generation_interface():
    class1 = widgets.IntSlider(
        value=12,
        min=0,
        max=1000,
        description='Class 1:'
    )
    
    class2 = widgets.IntSlider(
        value=200,
        min=0,
        max=1000,
        description='Class 2:'
    )
    
    truncation = widgets.FloatSlider(
        value=1.0,
        min=0.1,
        max=2.0,
        step=0.1,
        description='Truncation:'
    )
    
    return class1, class2, truncation

generator = ConditionalGeneration()
class1, class2, truncation = create_conditional_generation_interface()

def update_generation(class1, class2, truncation):
    # Генерируем базовые изображения
    z = torch.randn(2, 512).cuda()
    classes = torch.tensor([class1, class2], dtype=torch.int64).cuda()
    imgs = generator.generate_with_classes(z, classes)
    
    # Генерируем интерполяцию
    imgs_interp = generator.interpolate_classes(z[0], class1, class2)
    
    # Генерируем с усечением
    imgs_truncated = generator.generate_with_truncation(8, class1, truncation)
    
    plt.figure(figsize=(15, 5))
    
    plt.subplot(131)
    plt.imshow(to_image_grid(imgs))
    plt.title('Original Classes')
    plt.axis('off')
    
    plt.subplot(132)
    plt.imshow(to_image_grid(imgs_interp))
    plt.title('Class Interpolation')
    plt.axis('off')
    
    plt.subplot(133)
    plt.imshow(to_image_grid(imgs_truncated))
    plt.title(f'Truncation={truncation:.1f}')
    plt.axis('off')
    
    plt.tight_layout()

widgets.interactive(update_generation, class1=class1, class2=class2, truncation=truncation)

## 4. Манипуляции в латентном пространстве

В этом разделе мы исследуем возможности манипуляции латентными векторами для управления генерацией изображений.

In [ ]:
class LatentManipulator:
    """Класс для манипуляций в латентном пространстве."""
    
    def __init__(self, classifier_path='pretrained/regressor.pth'):
        self.regressor = CelebaAttributeClassifier('Smiling', classifier_path).cuda().eval()
        self.samples = []
        
    def collect_statistics(self, num_steps=200, batch_size=8):
        """Сбор статистики для обучения направления атрибута."""
        for latents in tqdm(torch.randn([num_steps, batch_size, 512])):
            with torch.no_grad():
                latents = G.style_gan2.style(latents.cuda())
                imgs = G(latents, w_space=True)
                probs = self.regressor.get_probs(preprocess(imgs))[:, 1]
                
            self.samples.extend([
                ShiftedGSample(l, p) for l, p in zip(latents.cpu(), probs.cpu())
            ])
            
    def find_attribute_direction(self, max_iter=10000):
        """Поиск направления атрибута с помощью SVR."""
        return train_normal(self.samples, max_iter)
    
    def apply_manipulation(self, z, direction, strength=5.0):
        """Применение манипуляции к латентным векторам."""
        with torch.no_grad():
            w = G.style_gan2.style(z)
            return G(w + strength * direction, w_space=True)
    
    def visualize_manipulation(self, z, direction, strengths=None):
        """Визуализация результатов манипуляции с разной силой.
        
        Args:
            z (torch.Tensor): Исходные латентные векторы
            direction (torch.Tensor): Направление манипуляции
            strengths (list): Список значений силы манипуляции
        """
        if strengths is None:
            strengths = [-10.0, -5.0, 0.0, 5.0, 10.0]
            
        results = [self.apply_manipulation(z, direction, s) for s in strengths]
        grid = torch.cat(results)
        
        plt.figure(figsize=(15, 3))
        plt.imshow(to_image_grid(grid, nrow=len(strengths)))
        plt.title('Attribute Manipulation')
        plt.axis('off')
        
        # Добавляем значения силы манипуляции под изображениями
        for i, s in enumerate(strengths):
            plt.text(i * grid.size(-1) / len(strengths), grid.size(-2) + 10,
                    f'λ={s:.1f}', ha='center')

### Интерактивный интерфейс для манипуляций

In [ ]:
def create_manipulation_interface():
    """Создание интерактивного интерфейса для манипуляций."""
    attribute = widgets.Dropdown(
        options=['Smiling', 'Young', 'Male', 'Glasses'],
        value='Smiling',
        description='Attribute:'
    )
    
    strength = widgets.FloatSlider(
        value=5.0,
        min=-10.0,
        max=10.0,
        step=0.5,
        description='Strength:'
    )
    
    random_seed = widgets.IntText(
        value=42,
        description='Seed:'
    )
    
    generate_button = widgets.Button(description='Generate')
    output = widgets.Output()
    
    return attribute, strength, random_seed, generate_button, output

manipulator = LatentManipulator()
attribute, strength, random_seed, generate_button, output = create_manipulation_interface()

def on_generate_click(_):
    with output:
        output.clear_output()
        
        # Устанавливаем seed для воспроизводимости
        torch.manual_seed(random_seed.value)
        
        # Собираем статистику если нужно
        if not manipulator.samples:
            print('Collecting statistics...')
            manipulator.collect_statistics()
            
        # Находим направление атрибута
        direction = manipulator.find_attribute_direction()
        
        # Генерируем и визуализируем результаты
        z = torch.randn(4, 512).cuda()
        manipulator.visualize_manipulation(z, direction, 
                                         [-strength.value, 0, strength.value])
        plt.show()

generate_button.on_click(on_generate_click)

# Отображаем интерфейс
widgets.VBox([attribute, strength, random_seed, generate_button, output])

## 5. Анализ качества генерации

В этом разделе мы проанализируем качество сгенерированных изображений с помощью различных метрик.

In [ ]:
class GenerationAnalyzer:
    """Класс для анализа качества генерации."""
    
    def __init__(self):
        self.lpips_model = lpips.LPIPS('alexnet').cuda()
        
    def compute_diversity(self, images, num_pairs=1000):
        """Вычисление метрики разнообразия на основе LPIPS."""
        n = len(images)
        pairs = torch.randperm(num_pairs * 2).view(-1, 2) % n
        distances = []
        
        for i, j in tqdm(pairs):
            with torch.no_grad():
                dist = self.lpips_model(images[i:i+1], images[j:j+1])
                distances.append(dist.item())
                
        return np.mean(distances), np.std(distances)
    
    def compute_attribute_consistency(self, images, attribute_classifier):
        """Вычисление консистентности атрибутов."""
        with torch.no_grad():
            probs = attribute_classifier.get_probs(preprocess(images))[:, 1]
            return probs.mean().item(), probs.std().item()
    
    def analyze_batch(self, images, attribute_classifier=None):
        """Полный анализ батча изображений."""
        diversity_mean, diversity_std = self.compute_diversity(images)
        
        results = {
            'diversity_mean': diversity_mean,
            'diversity_std': diversity_std
        }
        
        if attribute_classifier is not None:
            attr_mean, attr_std = self.compute_attribute_consistency(images, 
                                                                    attribute_classifier)
            results.update({
                'attribute_mean': attr_mean,
                'attribute_std': attr_std
            })
            
        return results

### Анализ качества генерации для разных параметров

In [ ]:
def analyze_generation_quality():
    analyzer = GenerationAnalyzer()
    results = []
    
    # Генерируем изображения с разными параметрами truncation
    truncations = [0.5, 1.0, 2.0]
    
    for trunc in truncations:
        print(f'\nAnalyzing truncation={trunc}:')
        
        # Генерируем батч изображений
        imgs = generator.generate_with_truncation(32, 239, trunc)
        
        # Анализируем качество
        metrics = analyzer.analyze_batch(imgs, manipulator.regressor)
        results.append((trunc, metrics))
        
        print(f'Diversity: {metrics["diversity_mean"]:.3f} ± {metrics["diversity_std"]:.3f}')
        if 'attribute_mean' in metrics:
            print(f'Attribute: {metrics["attribute_mean"]:.3f} ± {metrics["attribute_std"]:.3f}')
            
    return results

quality_results = analyze_generation_quality()

# Визуализируем результаты
plt.figure(figsize=(10, 5))

truncations = [r[0] for r in quality_results]
diversity = [r[1]['diversity_mean'] for r in quality_results]
attribute = [r[1].get('attribute_mean', 0) for r in quality_results]

plt.plot(truncations, diversity, 'b-', label='Diversity')
plt.plot(truncations, attribute, 'r-', label='Attribute')
plt.xlabel('Truncation')
plt.ylabel('Score')
plt.legend()
plt.grid(True)
plt.title('Generation Quality vs Truncation')
plt.show()

## Заключение

В этом ноутбуке мы:
1. Изучили метрики качества генерации (LPIPS)
2. Поработали с Deep Image Prior
3. Исследовали условную генерацию
4. Научились манипулировать латентным пространством
5. Проанализировали качество генерации

Основные выводы:
- LPIPS лучше соответствует человеческому восприятию качества изображений
- Deep Image Prior эффективен для задач реконструкции
- Манипуляции в латентном пространстве позволяют управлять атрибутами
- Существует компромисс между разнообразием и качеством генерации